In [47]:
# Find items starting with a publication or experiment set
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()
env = 'data'
my_auth = get_key('koray_data')   #use your key name in keypairs.json
schema_name = get_schema_names(my_auth) 

sets = ['942e5220-825d-4379-b9b8-e4d7a5098130', 'b51eb447-e41e-4fe1-9004-cd47392eda46']  # place the uuid of the item here
store, uuids = record_object_es(sets, my_auth, schema_name, add_pc_wfr=False)
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print(time2-time1)

64
64
3.759992837905884


In [48]:
transfer_env = 'fourfront-webdev'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)
# reverse lookup dictionary for schema names

rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]))
    for an_item in store[a_type]:
        counter += 1
        print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        # skip the items that exists
        if exists and existing:
            remove_existing_items.append(an_item['uuid'])
            print("{} {} can not post item".format(obj_type, an_item['uuid']))
            continue
 
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(len(second_round_items[a_type]))
    print()

    

user
8
1 986b362f-4eb6-4a9c-8173-3ab267307e3a
user 986b362f-4eb6-4a9c-8173-3ab267307e3a can not post item
2 7b8aa7bc-eee0-4bc4-9892-1c46bc4552c5
user 7b8aa7bc-eee0-4bc4-9892-1c46bc4552c5 can not post item
3 7c6184f0-3198-41d2-bb71-e7af0a9a74f0
user 7c6184f0-3198-41d2-bb71-e7af0a9a74f0 can not post item
4 32c70fcf-f3af-4687-89e2-9bdd05cf460a
user 32c70fcf-f3af-4687-89e2-9bdd05cf460a can not post item
5 900c5bf0-5b3a-4d7c-a6e6-7918faa19a77
user 900c5bf0-5b3a-4d7c-a6e6-7918faa19a77 can not post item
6 83b5073a-069b-4162-9b30-6f42d5551e34
user 83b5073a-069b-4162-9b30-6f42d5551e34 can not post item
7 b2ceeab8-b15b-4fdd-bcf7-49aaca674a63
user b2ceeab8-b15b-4fdd-bcf7-49aaca674a63 can not post item
8 986b362f-4eb6-4a9c-8173-3ab267111888
user 986b362f-4eb6-4a9c-8173-3ab267111888 can not post item
0

award
4
1 a725c2ce-5be2-4933-bf56-f0b7b3f1c584
award a725c2ce-5be2-4933-bf56-f0b7b3f1c584 can not post item
2 b0b9c607-f8b4-4f02-93f4-9895b461334b
award b0b9c607-f8b4-4f02-93f4-9895b461334b can not 

In [50]:
# Round II - patch the rest
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 

    for an_item in second_round_items[a_type]:
        counter += 1
        print(counter, an_item['uuid'])
        if 'attachment' in an_item.keys():
            del an_item['attachment']

        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)


user
award
lab
ontology
ontology_term
organism
target
6 1c9fcffc-0a2e-4c68-a5fc-1a50d6672a1a
vendor
7 57fa4e23-110d-459b-8eaf-3379b7d83177
protocol
8 37bfcdf6-1e12-4a91-816b-afff7c4b7cf4
biosample_cell_culture
9 da3a2406-6db1-4e6b-868f-2896d4d4f5fe
10 d870ef19-7fe7-483b-a590-908eeb79f7ec
individual_human
biosource
antibody
11 59460c2d-6e3b-4edf-9943-9312554212aa
biosample
12 98f3b166-d149-42f8-a4a1-68017e368fd9
13 63ee7a0b-5655-47de-9db9-d686cb449bfa
quality_metric_fastqc
14 eee9a01e-56c9-479b-b9e3-64f04723d085
15 e29a7999-dfd3-4daf-afef-f126a8c70f1e
16 d8477e2e-069e-4c71-b0df-4ac3e7539429
17 c311a759-b82d-4bd3-a0ce-602f5aa6175f
18 a604b3bf-685b-4ff0-97ac-d890db318f63
19 a04164d5-9a8e-4d17-8f13-a0298075774b
20 41cf65a1-16cc-4014-8749-ad99acd0da1a
21 032219c8-d9e0-483e-8f67-292a20a68c3b
file_fastq
22 bf1cbd5e-4eaf-4d0a-ba1c-b8cf758ff90b
23 9eddf3bb-fde3-4156-9b58-4689b22bacab
24 67d23c45-6a26-4de0-9518-f986081b8fef
25 65725bf4-d35e-443e-a1b0-b4bbe7e41d4c
26 4c3be0d1-cd00-4a14-85ed-43269

In [51]:
move_files = False

import boto3
s3 = boto3.resource('s3')

#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_auth)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']
print(source_att)

#target_addresses
target_health = ff_utils.get_metadata('/health', key = transfer_key)
target_raw = target_health['file_upload_bucket'] 
target_pf = target_health['processed_file_bucket'] 
target_att = target_health['blob_bucket'] 
print(target_att)

# # Round III - move attachments
# for a_type in my_types:
#     obj_type = rev_schema_name[a_type]
#     for an_item in second_round_items[a_type]:
#         if 'attachment' in an_item.keys():
#             at_key = an_item['attachment']['blob_id']
#             copy_source = {'Bucket': source_att, 'Key': at_key}
#             s3.meta.client.copy(copy_source, target_att, at_key)
#             print('attachment copied')




elasticbeanstalk-fourfront-webprod-blobs
elasticbeanstalk-fourfront-webdev-blobs


In [52]:
# Round IV - move files
for a_type in my_types:
    
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
    else:
        continue
        
    for an_item in second_round_items[a_type]:
        file_key = ff_utils.get_metadata(an_item['uuid'], key = my_auth)['upload_key']
        copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
        print(copy_source)
        try:
            s3.meta.client.copy(copy_source, target_file_bucket, file_key)
        except Exception as e:
            print(an_item['uuid'], 'can not be copied')
            raise e
        print('file copied')


{'Key': 'bf1cbd5e-4eaf-4d0a-ba1c-b8cf758ff90b/4DNFI89RWX1M.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '9eddf3bb-fde3-4156-9b58-4689b22bacab/4DNFIY13CD93.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '67d23c45-6a26-4de0-9518-f986081b8fef/4DNFI438ZP4U.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '65725bf4-d35e-443e-a1b0-b4bbe7e41d4c/4DNFINBXXW77.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '4c3be0d1-cd00-4a14-85ed-43269591fe41/4DNFIGM25CMN.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '38969bef-628a-41d6-bc74-c6b347c2e688/4DNFI43RP96B.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '35a1b573-9690-41b3-b0c0-0da8af5c55e8/4DNFI5JD6GGN.fastq.gz', 'Bucket': 'elasticbeanstalk-fourfront-webprod-files'}
file copied
{'Key': '26e6a9a9-defe-46e4-bdc6-7af4675bd17a/4D

In [44]:
print(source_health)

{'elasticsearch': 'search-fourfront-webprod2-fkav4x4wjvhgejtcg6ilrmczpe.us-east-1.es.amazonaws.com:80', 'blob_bucket': 'elasticbeanstalk-fourfront-webprod-blobs', '@context': '/health', 'load_data': 'encoded.loadxl:load_prod_data', 'database': 'fourfront-webprod.co3gwj7b7tpq.us-east-1.rds.amazonaws.com:5432/ebdb', 'beanstalk_env': 'fourfront-webprod2', '@id': '/health', 'display_title': 'Fourfront Status and Foursight Monitoring', 'ontology_updated': 'Never Generated', 'system_bucket': 'elasticbeanstalk-fourfront-webprod-system', 'content': None, '@type': ['Health', 'Portal'], 'file_upload_bucket': 'elasticbeanstalk-fourfront-webprod-files', 'processed_file_bucket': 'elasticbeanstalk-fourfront-webprod-wfoutput'}
